<b>openworkの口コミ情報スクレイピング用プログラム</b><br>
利用にはopenworkの会員IDとパスワードが必要です。<br>
<br>取得した企業データは2つのデータフレームに格納されます。<br><br>
企業の口コミデータ⇒df_company<br>
総合的な企業の評価点や、企業情報など⇒df_company_info


In [365]:
# ※注意※
# 以下の利用規約を熟読した上で、本プログラムの利用は自己責任でお願いします。
# 利用規約　OpenWork（旧:Vorkers）
# https://www.vorkers.com/rule.php

import warnings
warnings.filterwarnings('ignore')
import time
import datetime
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import numpy as np
from selenium.webdriver.chrome.options import Options # オプションを使うためには必要

# -------------------- 定数 -------------------- #
LOGIN_USER = "########"
LOGIN_PASS = "########"

#企業ID.任意の企業IDを指定する (リスト形式で複数指定も可能)
#一回の実行あたり、企業ID数は上限30までにすることを推奨(ブロック対策回避のため)
company_ids = [
'a0910000002WZ8v',

]

# https://www.vorkers.com/company_answer.php?m_id=xxxxxxxxxxxxxxx

kuchikomi_ids = [1]#(リスト形式で複数指定も可能)
# https://www.vorkers.com/company_answer.php?m_id=xxxxxxxxxxxxxxx&q_no=y
#   1   組織体制・企業文化
#   2   年収・給与制度
#   3   入社理由と入社後ギャップ
#   4   働きがい・成長
#   5   女性の働きやすさ
#   6   ワーク・ライフ・バランス
#   7   -
#   8   退職検討理由
#   9   企業分析［強み・弱み・展望］
#   10  経営者への提言

now = datetime.datetime.now()

DRIVER_PATH = './chromedriver'
BASE_URL = "https://www.vorkers.com/"

#wait time. いじらないことを推奨
SLEEP_TIME = 6.5

#指定したページ数(1ページ25コメント)だけコメントを取得する。
#(上限ページは多くても5程度が限度)
MAX_PAGES = 3

#Error Message. 要素を取得できない場合にNullで埋める
Error_Message = None
# --------------------------------------------- #
 
#option = Options()   # オプションを用意
#option.add_argument('--headless')   #headless mode にするかどうか
driver = webdriver.Chrome(executable_path = DRIVER_PATH)#options=option)
driver.get(BASE_URL) #Top pageにアクセス
time.sleep(SLEEP_TIME) #sleep timeを指定

# login page
driver.find_element_by_xpath('//*[@id="headerMenu"]/li[3]/a').click()
time.sleep(SLEEP_TIME)

# login
user_box = driver.find_element_by_xpath('//*[@id="_username"]')
pass_box = driver.find_element_by_xpath('//*[@id="_password"]')
user_box.send_keys(LOGIN_USER)
pass_box.send_keys(LOGIN_PASS)
driver.find_element_by_xpath('//*[@id="log_in"]').click()
time.sleep(SLEEP_TIME)

###スクレイピング項目の中身の入るハコを作る##
a=[]
b=[]
c=[]
d=[]
e=[]
f=[]
g=[]
h=[]

#こっちは企業情報が入るハコ
k0=[]
k1=[]
k2=[]
k3=[]
k4=[]
k5=[]
k6=[]
k7=[]
k8=[]
k9=[]
k10=[]
k11=[]
k12=[]
k13=[]
k14=[]

#####################

# scraping1 口コミ＋評点
for company_id in company_ids:
    #口コミIDごとに取得
    for kuchikomi_id in kuchikomi_ids:
        kuchikomi_start_url = BASE_URL + "company_answer.php?m_id=" + company_id + "&q_no=" + str(kuchikomi_id)

        page = 1
        
        while True:
            
            kuchikomi_now_url = kuchikomi_start_url + "&next_page=" + str(page)
            driver.get(kuchikomi_now_url)
            time.sleep(SLEEP_TIME)

            ######### エラー処理 #########
            # 最終ページ＋1ページ目は1ページ目に戻ってくる
            if driver.current_url == kuchikomi_start_url and page != 1:
                break

            # 1ページしかない場合は、2ページ目はエラー
            elif driver.title == "404エラー　File not found　OpenWork":
                break
                
            #指定したページ数に達したら終了
            elif page > MAX_PAGES:
                break
            ###########################

            ##### スクレイピング処理 #####
            ####  口コミが1件も投稿されていない企業に関しては、find_element_by_xpath()が
            ####  NoSuchElementExceptionとなり、tryの中でbreakされるため、
            ####  スクレイピング処理自体行われていない      ####
            print("[---------- " + driver.title + " ----------]")
            # articleは1ページ最大25件
            for i in range(1, 26):
                
                print("[ article " + str(i) + " ]")

                # 1. 回答日（2018年05月17日）
                answer_time_xpath  = '//*[@id="anchor01"]/article[' + str(i) + ']/div[1]/p/time'
                # 2. 回答者属性（SE、在籍3年未満、現職（回答時）、新卒入社、男性、xxx）
                answer_type_xpath  = '//*[@id="anchor01"]/article[' + str(i) + ']/div[2]/dl/dt/a'
                # 3. 点数（2.4）
                answer_score_xpath = '//*[@id="anchor01"]/article[' + str(i) + ']/div[2]/dl/dt/span[2]/span[6]'
                # 4. 本文
                answer_body_xpath  = '//*[@id="anchor01"]/article[' + str(i) + ']/div[2]/dl/dd'
                # 5 業種
                company_type_xpath = '//*[@id="jsCompanyInfoBox"]/div[2]/dl/dd[1]/ul/li/a'
                # 6 企業名
                company_name_xpath = '//*[@id="anchor01"]/div[1]/h3'

                try:
                    answer_time_text  = driver.find_element_by_xpath(answer_time_xpath).text
                    answer_type_text  = driver.find_element_by_xpath(answer_type_xpath).text
                    answer_score_text = driver.find_element_by_xpath(answer_score_xpath).text
                    answer_body_text  = driver.find_element_by_xpath(answer_body_xpath).text
                    company_type_text = driver.find_element_by_xpath(company_type_xpath).text
                    company_name_text = driver.find_element_by_xpath(company_name_xpath).text
                # articleが25件未満
                except Exception:
                # except selenium.common.exceptions.NoSuchElementException:
                    break
                
                #空リストに値を挿入
                a+=[company_name_text]
                b+=[company_id]
                c+=[str(kuchikomi_id)]
                d+=[answer_time_text]
                e+=[answer_type_text]
                f+=[answer_score_text]
                g+=[answer_body_text]
                h+=[company_type_text]
                
                #with open(OUTPUT_CSV_FILEPATH, mode='a',encoding='utf_8_sig') as f:
                    #f.write(tmp_str)
            ###########################

            page += 1
    
    # scraping2 企業情報
    kuchikomi_top_url = BASE_URL + "company.php?m_id=" + company_id
    driver.get(kuchikomi_top_url)
    time.sleep(SLEEP_TIME)
    print('--TOP page--')
    #0. 企業名
    companyname_xpath = '//*[@id="firstArticleTitle"]/div/div/h3'
    # 1. 残業時間
    overtime_xpath  = '//*[@id="mainColumn"]/article[1]/div[2]/div[1]/div[2]/dl[1]/dd/span'
    #2. 有給消化率
    paidvacation_xpath = '//*[@id="mainColumn"]/article[1]/div[2]/div[1]/div[2]/dl[2]/dd/span'
    #3. 平均年収
    average_salary_xpath = '//*[@id="mainColumn"]/article[3]/div[2]/table/tbody/tr[1]/td/span[1]'
    #4. 設立年
    #エキスパンドのxpath
    expand_xpath = '//*[@id="jsCompanyInfoBox"]/p/a'
    #「もっとみる」をクリックしてエキスパンド
    driver.execute_script('arguments[0].click();', driver.find_element_by_xpath(expand_xpath))
    #Wait
    #ここのWait時間は短くて構わない
    time.sleep(1)
        
    #設立年のxpathが異なるケースがあるため、分岐させる
    if '年' in driver.find_element_by_xpath('//*[@id="jsCompanyInfoBox"]/div[2]/dl/dd[5]').text:
        established_year_xpath = '//*[@id="jsCompanyInfoBox"]/div[2]/dl/dd[5]'
    elif '年' in driver.find_element_by_xpath('//*[@id="jsCompanyInfoBox"]/div[2]/dl/dd[4]').text:
        established_year_xpath = '//*[@id="jsCompanyInfoBox"]/div[2]/dl/dd[4]'
    #そもそも年が存在しないため、取得できない場合は、業種をダミーデータで入れとく
    else:
        established_year_xpath = '//*[@id="jsCompanyInfoBox"]/div[2]/dl/dd[1]'
            
    #5.総合スコア
    totalscore_xpath = '//*[@id="mainColumn"]/article[1]/div[2]/div[1]/div[1]/p[2]/span'
        
    #6.社員の士気
    motivation_xpath = '//*[@id="mainColumn"]/article[1]/div[2]/div[2]/ul/li[2]/dl/dd'
        
    #7.風通し良さ
    openness_xpath = '//*[@id="mainColumn"]/article[1]/div[2]/div[2]/ul/li[3]/dl/dd'

    #8.社員相互尊重
    respect_xpath = '//*[@id="mainColumn"]/article[1]/div[2]/div[2]/ul/li[4]/dl/dd'
        
    #9.20代成長性
    twenties_xpath = '//*[@id="mainColumn"]/article[1]/div[2]/div[2]/ul/li[5]/dl/dd'
        
    #10.人材長期育成
    longterm_xpath = '//*[@id="mainColumn"]/article[1]/div[2]/div[2]/ul/li[6]/dl/dd'
        
    #11.法令遵守
    compliance_xpath = '//*[@id="mainColumn"]/article[1]/div[2]/div[2]/ul/li[7]/dl/dd'
        
    #12.人事評価適正感
    fairness_xpath = '//*[@id="mainColumn"]/article[1]/div[2]/div[2]/ul/li[7]/dl/dd'
        
    #13. 待遇満足度
    salary_xpath = '/html/body/div[1]/main/div/div[1]/article[1]/div[2]/div[2]/ul/li[1]/dl/dd'  
    
    #14. 業種
    company_type_xpath = '//*[@id="jsCompanyInfoBox"]/div[2]/dl/dd[1]/ul/li/a'
    
        
    #テキスト取得(エレメントが存在しない場合があるため、全要素において例外処理をいれる)
    try:
        company_name_text = driver.find_element_by_xpath(companyname_xpath).text
    except Exception:
        company_name_text = Error_Message
    
    try:
        overtime_text = driver.find_element_by_xpath(overtime_xpath).text
    except Exception:
        overtime_text = Error_Message
        
    try:
        paidvacation_text  = driver.find_element_by_xpath(paidvacation_xpath).text
    except Exception:
        paidvacation_text = Error_Message
    
    try:
        average_salary_text = driver.find_element_by_xpath(average_salary_xpath).text
    except Exception:
        average_salary_text = Error_Message
            
    #設立年はトライ処理必要ない
    established_year_text = driver.find_element_by_xpath(established_year_xpath).text

    try:
        totalscore_text = driver.find_element_by_xpath(totalscore_xpath).text
    except Exception:
        totalscore_text = Error_Message
            
    try:
        motivation_text = driver.find_element_by_xpath(motivation_xpath).text
    except Exception:
        motivation_text = Error_Message
            
    try:
        openness_text = driver.find_element_by_xpath(openness_xpath).text
    except Exception:
        openness_text = Error_Message
            
    try:
        respect_text = driver.find_element_by_xpath(respect_xpath).text
    except Exception:
        respect_text = Error_Message
            
    try:
        twenties_text = driver.find_element_by_xpath(twenties_xpath).text
    except Exception:
        twenties_text = Error_Message
            
    try:
        longterm_text = driver.find_element_by_xpath(longterm_xpath).text
    except Exception:
        longterm_text = Error_Message
        
    try:
        compliance_text = driver.find_element_by_xpath(compliance_xpath).text
    except Exception:
        compliance_text = Error_Message
        
    try:
        fairness_text = driver.find_element_by_xpath(fairness_xpath).text
    except Exception:
        fairness_text = Error_Message
        
    try:
        salary_text = driver.find_element_by_xpath(salary_xpath).text
    except Exception:
        salary_text = Error_Message
        
    try:
        comp_type_text = driver.find_element_by_xpath(company_type_xpath).text
    except Exception:
        comp_type_text = Error_Message
        

    #空リストに値代入
    k0.append(company_name_text)
    
    #月残業時間が'--'で取得される場合があるので、その場合はk4にNoneを追加
    if not '--' in overtime_text:
        k1.append(overtime_text)
    else:
        k1.append(None)
        
    #有給消化率も同様の処理を入れる
    if not '--' in paidvacation_text:
        k2.append(paidvacation_text)
    else:
        k2.append(None)
    #平均年収を代入
    k3.append(average_salary_text)
        
    #established_yearが設立年を正しく取れていないケースがあるので、取れていない場合はk4にNoneを追加
    #正しく取れているのであれば、「年」を除去した上で、int型に直してからk4に追加
    if '年' in established_year_text:
        established_year_text = int(established_year_text.replace('年',''))
        k4.append(established_year_text)
    else:
        k4.append(None)
    k5.append(totalscore_text)
    k6.append(motivation_text)
    k7.append(openness_text)
    k8.append(respect_text)
    k9.append(twenties_text)
    k10.append(longterm_text)
    k11.append(compliance_text)
    k12.append(fairness_text)
    k13.append(salary_text)
    k14.append(comp_type_text)

            
#quit
driver.quit()

#ここから、それぞれの収集したデータをデータフレームにする
#df_company_infoのハコ
df_company = pd.DataFrame(data={'企業名':a, '企業ID': b, '口コミ種別ID': c,'回答日時': d, '回答タイプ': e,'評価':f,'本文':g,'業種':h},columns=['企業名', '企業ID', '口コミ種別ID', '回答日時', '回答タイプ','評価','本文','業種'])

#df_company_infoのハコ
df_company_info = pd.DataFrame(index=[],columns=['企業名','月残業時間','有給消化率','平均年収','設立年','総合スコア',
                                    '社員の士気','風通し','社員相互尊重',
                                    '20代成長性','長期育成','コンプラ',
                                    '評価納得感','待遇満足度','業種'])

#company_nameが入っているリストk0の要素長さ分だけ、dataframeへappendを行う
for i in range(len(k0)):
    df_company_info = df_company_info.append({'企業名': k0[i], '月残業時間': k1[i],
                            '有給消化率':k2[i],
                            '平均年収':k3[i],'設立年':k4[i],'総合スコア':k5[i],
                            '社員の士気':k6[i],'風通し':k7[i],'社員相互尊重':k8[i],
                            '20代成長性':k9[i],'長期育成':k10[i],'コンプラ':k11[i],
                            '評価納得感':k12[i],'待遇満足度':k13[i],'業種':k14[i]
                            },ignore_index=True)

#df_companyに回答種別をくっつける###
answer_type={'1':"組織体制・企業文化",'2':"年収・給与制度",'3':"入社理由と入社後ギャップ",'4':"働きがい・成長",
             '5':"女性の働きやすさ",'6':"ワーク・ライフ・バランス",'8':"退職検討理由",'9':"企業分析［強み・弱み・展望］",
            '10':"経営者への提言"}

df_company["口コミ種別"] = df_company.口コミ種別ID.map(answer_type)
##############################

####回答タイプから、職種、在籍年数、現職かどうか、新卒かどうか、性別を抜き出す
job_type = df_company["回答タイプ"].str.split(("、"),expand=True).iloc[:,0]
years = df_company["回答タイプ"].str.split(("、"),expand=True).iloc[:,1]
stay = df_company["回答タイプ"].str.split(("、"),expand=True).iloc[:,2]
new_graduate = df_company["回答タイプ"].str.split(("、"),expand=True).iloc[:,3]
sex = df_company["回答タイプ"].str.split(("、"),expand=True).iloc[:,4]

#df_companyに新しい列として追加
df_company['職種'] = job_type
df_company['経験年数'] = years
df_company['現職/退職'] = stay
df_company['新卒/中途'] = new_graduate
df_company['性別'] = sex

#元の回答タイプのカラムはいらないので落とす
df_company.drop("回答タイプ",axis=1,inplace=True)

#本文の改行コード削除
df_company.本文 = df_company.本文.replace('\n','',regex=True)

#年収をint、月残業時間をfloatに、有給消化率はfloatに直したいが、取得できなかった場合はNoneが入っているので
#該当するカラムがNoneではない行に対してのみ、置換を行う
df_company_info[df_company_info['平均年収'].notna()].平均年収 = df_company_info[df_company_info['平均年収'].notna()].平均年収.astype('int')
df_company_info[df_company_info['月残業時間'].notna()].月残業時間 = df_company_info[df_company_info['月残業時間'].notna()].月残業時間.astype('float')
df_company_info[df_company_info['有給消化率'].notna()].有給消化率 = df_company_info[df_company_info['有給消化率'].notna()].有給消化率.astype('float')

####結果のshape確認
print(df_company.shape)
print(df_company_info.shape)

[---------- セールスフォース・ジャパン（旧：セールスフォース・ドットコム）の「組織体制・企業文化」 OpenWork ----------]
[ article 1 ]
[ article 2 ]
[ article 3 ]
[---------- セールスフォース・ジャパン（旧：セールスフォース・ドットコム）の「組織体制・企業文化」 2ページ目 OpenWork ----------]
[ article 1 ]
[ article 2 ]
[ article 3 ]
[ article 4 ]
[ article 5 ]
[ article 6 ]
[ article 7 ]
[ article 8 ]
[ article 9 ]
[ article 10 ]
[ article 11 ]
[ article 12 ]
[ article 13 ]
[ article 14 ]
[ article 15 ]
[ article 16 ]
[ article 17 ]
[ article 18 ]
[ article 19 ]
[ article 20 ]
[ article 21 ]
[ article 22 ]
[ article 23 ]
[ article 24 ]
[ article 25 ]
[---------- セールスフォース・ジャパン（旧：セールスフォース・ドットコム）の「組織体制・企業文化」 3ページ目 OpenWork ----------]
[ article 1 ]
[ article 2 ]
[ article 3 ]
--TOP page--
(29, 13)
(1, 15)


In [366]:
#口コミ件数が5件に満たない企業のデータは,company_dfからdropする
def drop_fewreviews(df):
    omit_list = []
    apd = pd.DataFrame(df_company.groupby('企業名').本文.count() < 5)
    for a in apd[apd['本文'] == True].index:
        omit_list.append(a)
    for b in omit_list:
        df_company.drop(df_company.loc[df_company.企業名==b].index,axis=0, inplace=True)
        
    display(omit_list)
    display(df_company.企業名.unique())
    
drop_fewreviews(df_company)

[]

array(['株式会社セールスフォース・ジャパン（旧：株式会社セールスフォース・ドットコム）'], dtype=object)

In [367]:
#正しく取得できているかを確認
display(df_company.head(20))
display(df_company_info.head(25))

,企業名,企業ID,口コミ種別ID,回答日時,評価,本文,業種,口コミ種別,職種,経験年数,現職/退職,新卒/中途,性別
0,株式会社セールスフォース・ジャパン（旧：株式会社セールスフォース・ドットコム）,a0910000002WZ8v,1,2021年08月19日,4.0,組織は、担当するお客様企業の従業員数によって分かれています。10名以下のベンチャー企業を担当...,SIer、ソフト開発、システム運用,組織体制・企業文化,SE,在籍3年未満,現職（回答時）,中途入社,女性
1,株式会社セールスフォース・ジャパン（旧：株式会社セールスフォース・ドットコム）,a0910000002WZ8v,1,2021年08月07日,4.8,お互い助け合う文化があると思います。また雰囲気は明るく、オフィスも素敵でやる気に溢れる環境だ...,SIer、ソフト開発、システム運用,組織体制・企業文化,営業,在籍3～5年,現職（回答時）,中途入社,男性
2,株式会社セールスフォース・ジャパン（旧：株式会社セールスフォース・ドットコム）,a0910000002WZ8v,1,2021年08月13日,4.9,"カスタマーサクセスの考え方が深く浸透しており、営業部門としていわゆる""売り逃げ""を悪とする想...",SIer、ソフト開発、システム運用,組織体制・企業文化,営業,Account Executive,在籍3～5年,現職（回答時）,中途入社
3,株式会社セールスフォース・ジャパン（旧：株式会社セールスフォース・ドットコム）,a0910000002WZ8v,1,2021年07月29日,3.8,対外的にはOhana文化、カスタマーサクセス等きらびやかなメッセージが発信され、それがブラン...,SIer、ソフト開発、システム運用,組織体制・企業文化,営業,在籍3年未満,現職（回答時）,中途入社,男性
4,株式会社セールスフォース・ジャパン（旧：株式会社セールスフォース・ドットコム）,a0910000002WZ8v,1,2021年06月19日,3.8,社員やステークホルダーを家族として迎える風土が根付いている。過去の導入事例が多いく、成功事例...,SIer、ソフト開発、システム運用,組織体制・企業文化,AE,在籍3年未満,退社済み(2020年以降),中途入社,男性
5,株式会社セールスフォース・ジャパン（旧：株式会社セールスフォース・ドットコム）,a0910000002WZ8v,1,2021年03月09日,4.9,チームセリングの意識が高く、聞かれれば誰でも回答してくれる文化。その上で数字には厳格で皆が数...,SIer、ソフト開発、システム運用,組織体制・企業文化,西日本支社,営業,営業担当,在籍5～10年,現職（回答時）
6,株式会社セールスフォース・ジャパン（旧：株式会社セールスフォース・ドットコム）,a0910000002WZ8v,1,2021年03月09日,3.5,企業としては全てのステークホルダーを家族のように想い助け合う、Ohanaカルチャーと呼ばれる...,SIer、ソフト開発、システム運用,組織体制・企業文化,アカウントエグゼクティブ,在籍3～5年,現職（回答時）,中途入社,男性
7,株式会社セールスフォース・ジャパン（旧：株式会社セールスフォース・ドットコム）,a0910000002WZ8v,1,2021年01月29日,3.9,自由に発言でき、若いメンバーが多いためか仲がよく、助け合える会社は常に先進的な方向に進んでお...,SIer、ソフト開発、システム運用,組織体制・企業文化,エンタープライズ,営業,アカウントエグゼクティブ,在籍3～5年,現職（回答時）
8,株式会社セールスフォース・ジャパン（旧：株式会社セールスフォース・ドットコム）,a0910000002WZ8v,1,2021年01月07日,3.1,どうしても大企業化してきているため業務範囲も細分化されてきており、また古くからいる社員が最も...,SIer、ソフト開発、システム運用,組織体制・企業文化,管理部門,在籍3年未満,現職（回答時）,中途入社,女性
9,株式会社セールスフォース・ジャパン（旧：株式会社セールスフォース・ドットコム）,a0910000002WZ8v,1,2020年12月22日,4.3,非常に風通しがよく、また社会にも貢献しようという魅力的な企業文化を有している。社員も前向きで...,SIer、ソフト開発、システム運用,組織体制・企業文化,カスタマーサクセス,サービス,ディレクター,在籍3～5年,退社済み(2020年より前)


,企業名,月残業時間,有給消化率,平均年収,設立年,総合スコア,社員の士気,風通し,社員相互尊重,20代成長性,長期育成,コンプラ,評価納得感,待遇満足度,業種
0,株式会社セールスフォース・ジャパン（旧：株式会社セールスフォース・ドットコム）,39.3,58.9,1223,2000,4.65,4.6,4.5,4.1,4.6,3.8,4.6,4.6,4.6,SIer、ソフト開発、システム運用


In [368]:
#SqLiteに書き込む
import sys
import sqlite3

sys.stderr.write("*** 開始 ***\n")

#書き込み先db指定
file_company = "./datas/OpenWork_company.db"
#file_company_info = './datas/OpenWork_company_info.db'

#接続
conn = sqlite3.connect(file_company)
conn2 = sqlite3.connect(file_company_info)
#
#df_companyをsqlに追加
df_company.to_sql('df_company',conn,if_exists='append',index=None)
df_company_info.to_sql('df_company_info',conn2,if_exists='append',index=None)
#
conn.close()
conn2.close()
#
sys.stderr.write("*** 終了 ***\n")

*** 開始 ***
*** 終了 ***


11

In [369]:
#SQLiteからの読み込み
import sys
import pandas as pd
#
import sqlite3
# ------------------------------------------------------------------
sys.stderr.write("*** 開始 ***\n")
#読み込み先
file_company = "./datas/OpenWork_company.db"
file_company_info = './datas/OpenWork_company_info.db'

conn = sqlite3.connect(file_company)
conn2 = sqlite3.connect(file_company_info)
#
df_company=pd.read_sql_query('SELECT * FROM df_company', conn)
df_company_info=pd.read_sql_query('SELECT * FROM df_company_info', conn2)
display(df_company.head(15))
display(df_company_info.head(25))
#
conn.close()
conn2.close()
sys.stderr.write("*** 終了 ***\n")

*** 開始 ***


,企業名,企業ID,口コミ種別ID,回答日時,評価,本文,業種,口コミ種別,職種,経験年数,現職/退職,新卒/中途,性別
0,株式会社極洋,a0910000000Fqdi,1,2021年09月16日,2.9,典型的な昔の会社。上の人間になればなるほど安定を好み、チャレンジ精神や新しい文化を取り入れる...,食品、飲料,組織体制・企業文化,水産,営業,在籍3年未満,現職（回答時）,新卒入社
1,株式会社極洋,a0910000000Fqdi,1,2018年04月12日,3.8,一部上場企業なので土台はしっかりしている。歴史も80年以上あるので、それだけ続いてきた会社な...,食品、飲料,組織体制・企業文化,管理部門,在籍3～5年,退社済み(2015年より前),新卒入社,女性
2,株式会社極洋,a0910000000Fqdi,1,2017年05月17日,2.8,水産、加工食品、常温、鰹鮪の４事業が主力。本社、支社に分けられる。支社が販社、独立採算制。人...,食品、飲料,組織体制・企業文化,営業,在籍10～15年,退社済み(2020年より前),新卒入社,男性
3,株式会社極洋,a0910000000Fqdi,1,2018年03月05日,3.3,良い意味では会社が古く、福利厚生・給与等安定している。また、基本的にクビにもならない。悪い意...,食品、飲料,組織体制・企業文化,社員,在籍3～5年,現職（回答時）,新卒入社,男性
4,株式会社極洋,a0910000000Fqdi,1,2018年01月04日,2.6,古い会社なので年功序列、上の言うことは絶対の風潮があり、風通しは悪い。また年齢構成がいびつに...,食品、飲料,組織体制・企業文化,営業,在籍5～10年,現職（回答時）,新卒入社,男性
5,株式会社極洋,a0910000000Fqdi,1,2015年07月17日,3.3,水産気質というか、たくましい人が多い。上下関係は厳しかったが、根が優しい人が多かった印象。実...,食品、飲料,組織体制・企業文化,営業,在籍3～5年,退社済み(2015年より前),新卒入社,男性
6,日本水産株式会社,a0910000000Fqdk,1,2016年03月25日,2.0,終身雇用・年功序列・新卒偏重・男性社会などの昭和の遺産から脱却できていない。それらに当てはま...,食品、飲料,組織体制・企業文化,事務,在籍5～10年,現職（回答時）,中途入社,女性
7,日本水産株式会社,a0910000000Fqdk,1,2017年04月12日,2.9,体育会系かと思います。お客様が第一、上司が第一。食品と水産とファイン事業があるので部署により...,食品、飲料,組織体制・企業文化,マーケティング,在籍5～10年,現職（回答時）,新卒入社,女性
8,日本水産株式会社,a0910000000Fqdk,1,2016年09月19日,2.4,社長が交代して変わった可能性があるが、「臭いものには蓋をする」という風土があったと思う。何か...,食品、飲料,組織体制・企業文化,管理部門,在籍3～5年,退社済み(2015年より前),中途入社,女性
9,日本水産株式会社,a0910000000Fqdk,1,2016年03月03日,2.8,創業100年を超えている老舗企業であるので、組織体制及び企業文化については良くも悪くも老舗の...,食品、飲料,組織体制・企業文化,営業,在籍10～15年,退社済み(2015年より前),新卒入社,男性


,企業名,月残業時間,有給消化率,平均年収,設立年,総合スコア,社員の士気,風通し,社員相互尊重,20代成長性,長期育成,コンプラ,評価納得感,待遇満足度,業種
0,シークス,16.9,36.9,None,1937.0,3.00,3.3,2.9,2.4,3.1,2.5,3.2,3.2,3.5,食品、飲料
1,日本水産株式会社,31.1,59.0,None,1943.0,2.98,2.9,2.9,2.5,2.6,2.3,4.1,4.1,3.5,食品、飲料
2,マルハニチロ株式会社,16.2,49.3,530,1943.0,3.15,2.9,3.1,2.9,3.2,2.9,4.4,4.4,2.9,食品、飲料
3,株式会社ホウスイ,None,None,None,1945.0,2.93,2.9,2.9,2.9,3.0,2.8,2.9,2.9,3.2,食品、飲料
4,株式会社雪国まいたけ,14.3,46.6,None,2015.0,2.48,2.4,2.4,2.5,2.2,2.3,2.7,2.7,2.7,食品、飲料
5,カネコ種苗株式会社,20.2,20.0,None,1947.0,2.76,2.8,2.4,2.8,2.5,2.2,3.6,3.6,2.8,農業、林業、水産、畜産
6,株式会社サカタのタネ,10.7,54.8,None,1942.0,3.20,2.8,3.6,3.1,2.6,2.9,4.0,4.0,3.6,農業、林業、水産、畜産
7,ホクト株式会社（食品）,21.4,58.2,423,1964.0,2.85,3.0,2.7,3.4,2.6,2.6,3.2,3.2,2.7,食品、飲料
8,株式会社ホクリヨウ,29.0,37.0,None,1949.0,2.89,2.9,2.9,2.8,2.7,2.9,3.0,3.0,3.0,食品、飲料
9,株式会社ヒノキヤグループ,32.2,39.6,None,1988.0,2.89,2.9,3.0,2.9,2.9,2.9,3.0,3.0,2.7,不動産関連、住宅


*** 終了 ***


11

In [370]:
df_company_info.tail()

,企業名,月残業時間,有給消化率,平均年収,設立年,総合スコア,社員の士気,風通し,社員相互尊重,20代成長性,長期育成,コンプラ,評価納得感,待遇満足度,業種
2212,アマゾンジャパン合同会社,28.8,69.8,1004,2000.0,3.88,3.8,4.0,3.1,3.6,2.4,4.4,4.4,4.2,インターネット
2213,日本マイクロソフト株式会社,36.8,60.6,1290,1986.0,4.22,4.1,4.2,3.8,3.7,2.7,4.8,4.8,4.6,SIer、ソフト開発、システム運用
2214,アクセンチュア株式会社,39.0,66.2,854,1995.0,4.24,4.2,4.3,3.9,4.6,3.0,3.8,3.8,4.1,コンサルティング、シンクタンク
2215,日本アイ・ビー・エム株式会社（IBM）,38.9,49.8,915,1937.0,4.02,3.5,4.2,3.7,4.0,3.2,4.6,4.6,3.4,SIer、ソフト開発、システム運用
2216,株式会社セールスフォース・ジャパン（旧：株式会社セールスフォース・ドットコム）,39.3,58.9,1223,2000.0,4.65,4.6,4.5,4.1,4.6,3.8,4.6,4.6,4.6,SIer、ソフト開発、システム運用


In [371]:
df_company.tail(90)

,企業名,企業ID,口コミ種別ID,回答日時,評価,本文,業種,口コミ種別,職種,経験年数,現職/退職,新卒/中途,性別
39655,日本マイクロソフト株式会社,a0910000000GVqH,1,2017年06月14日,4.6,良い意味で外資系と日系の中間に位置するような文化を持っています。基本的に実力主義なので、優秀...,SIer、ソフト開発、システム運用,組織体制・企業文化,サポートエンジニア,在籍3～5年,現職（回答時）,中途入社,男性
39656,日本マイクロソフト株式会社,a0910000000GVqH,1,2017年05月16日,3.0,「個人商店」個人の自由度は高く、出社・退社時間、座席、PCの管理、研修受講のタイミング等はか...,SIer、ソフト開発、システム運用,組織体制・企業文化,プレミアフィールドエンジニア,在籍3～5年,現職（回答時）,中途入社,男性
39657,日本マイクロソフト株式会社,a0910000000GVqH,1,2017年04月07日,2.5,基本的な組織の骨組みは事業部毎の縦割り。会社として目指す未来像は素晴らしいものであるが、企業...,SIer、ソフト開発、システム運用,組織体制・企業文化,Sales（Account Executive）,在籍10～15年,現職（回答時）,中途入社,男性
39658,アクセンチュア株式会社,a0910000000G33n,1,2022年02月26日,4.3,組織が巨大なため組織体制はかなり細分化されており、入社時のポジション・部門・業界によって業務...,コンサルティング、シンクタンク,組織体制・企業文化,テクノロジーコンサルティング本部,ビジネスコンサルタント,シニアアナリスト,在籍3年未満,現職（回答時）
39659,アクセンチュア株式会社,a0910000000G33n,1,2022年01月15日,4.5,人と人との馴れ合いではなく、論理的に意見を言い、バリューを出している人間がそのまま評価されま...,コンサルティング、シンクタンク,組織体制・企業文化,テクノロジー本部,ITコンサルタント,マネージャ,在籍15～20年,退社済み(2020年以降)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39740,株式会社セールスフォース・ジャパン（旧：株式会社セールスフォース・ドットコム）,a0910000002WZ8v,1,2021年06月11日,3.4,成果主義であり、成果を賞賛しそれをサポートする風土は根強く、周りの方の成果を必ず褒めて讃える...,SIer、ソフト開発、システム運用,組織体制・企業文化,営業,在籍3年未満,現職（回答時）,中途入社,男性
39741,株式会社セールスフォース・ジャパン（旧：株式会社セールスフォース・ドットコム）,a0910000002WZ8v,1,2021年04月03日,3.6,営業変革を自分たちで実践しながら、そのノウハウを顧客に使えて変革するよく出来たモデルをベース...,SIer、ソフト開発、システム運用,組織体制・企業文化,エンタ,営業,AE,在籍3年未満,退社済み(2020年より前)
39742,株式会社セールスフォース・ジャパン（旧：株式会社セールスフォース・ドットコム）,a0910000002WZ8v,1,2021年01月21日,2.5,非常に体育会｡ポジティブであることが正義｡普通の日本の営業会社です｡データを元に戦略的に営業...,SIer、ソフト開発、システム運用,組織体制・企業文化,インサイドセールス,在籍3年未満,現職（回答時）,中途入社,女性
39743,株式会社セールスフォース・ジャパン（旧：株式会社セールスフォース・ドットコム）,a0910000002WZ8v,1,2020年12月31日,3.8,オハナ文化は会社に浸透しているようで、部門によって大きな差がある。EBUとCBUでも雰囲気が...,SIer、ソフト開発、システム運用,組織体制・企業文化,営業,在籍3～5年,退社済み(2020年以降),中途入社,男性


In [372]:
len(df_company.企業名.unique())

1493

In [373]:
len(df_company_info.企業名.unique())

2172

In [374]:
df_company.duplicated().sum()

452

In [375]:
df_company_info.duplicated().sum()

45